# Loaddata

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('weather.csv')

X = data.drop('play', axis=1)
y = data['play']
y = LabelEncoder().fit_transform(y)

X,y

(     outlook temperature humidity  windy
 0   overcast         hot     high  False
 1   overcast        cool   normal   True
 2   overcast        mild     high   True
 3   overcast         hot   normal  False
 4      rainy        mild     high  False
 5      rainy        cool   normal  False
 6      rainy        cool   normal   True
 7      rainy        mild   normal  False
 8      rainy        mild     high   True
 9      sunny         hot     high  False
 10     sunny         hot     high   True
 11     sunny        mild     high  False
 12     sunny        cool   normal  False
 13     sunny        mild   normal   True,
 array([1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1]))

# init func entropy informatine_gain

In [5]:
def entropy(y):
    """
    Tính entropy của một tập dữ liệu nhãn y.
    """
    classes = np.unique(y)
    entropy = 0
    for cls in classes:
        p_cls = np.sum(y == cls) / len(y)
        entropy -= p_cls * np.log2(p_cls)
    return entropy

def information_gain(X, y, feature):
    """
    Tính lợi ích thông tin khi chia tập dữ liệu dựa trên một đặc trưng cụ thể.
    """
    # Tính entropy của tập dữ liệu gốc
    original_entropy = entropy(y)
    
    # Tính entropy sau khi chia tập dữ liệu dựa trên đặc trưng
    values = np.unique(X[feature])
    new_entropy = 0
    for value in values:
        subset_y = y[X[feature] == value]
        weight = len(subset_y) / len(y)
        new_entropy += weight * entropy(subset_y)
    
    # Tính lợi ích thông tin
    information_gain = original_entropy - new_entropy
    return information_gain


# Buil tree

In [6]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature = 0
        self.threshold = 0
        self.left = None
        self.right = None

def build_tree(X, y, depth=0, max_depth=None):
    """
    Xây dựng cây quyết định dựa trên dữ liệu huấn luyện và nhãn tương ứng.
    """
    # Kiểm tra điều kiện dừng
    if depth == max_depth:
        return Node(predicted_class=np.argmax(np.bincount(y)))
    
    if len(np.unique(y)) == 1:
        return Node(predicted_class=y[0])
    
    # Chọn đặc trưng và ngưỡng tốt nhất để chia
    best_feature = None
    best_gain = -1
    for feature in X.columns:
        gain = information_gain(X, y, feature)
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
    
    # Tạo node và chia tập dữ liệu
    node = Node(predicted_class=np.argmax(np.bincount(y)))
    node.feature = best_feature
    
    unique_values = np.unique(X[best_feature])
    if len(unique_values) == 1:  # Không thể chia tiếp
        return node
    
    # Tiếp tục chia dữ liệu cho các nhánh con
    if isinstance(unique_values[0], str):  # Đối với dữ liệu rời rạc
        left_idxs = X[best_feature] == unique_values[0]
        right_idxs = X[best_feature] != unique_values[0]
    else:  # Đối với dữ liệu liên tục
        threshold = np.median(unique_values)
        left_idxs = X[best_feature] <= threshold
        right_idxs = X[best_feature] > threshold
    
    node.threshold = threshold if not isinstance(unique_values[0], str) else None
    node.left = build_tree(X[left_idxs], y[left_idxs], depth + 1, max_depth)
    node.right = build_tree(X[right_idxs], y[right_idxs], depth + 1, max_depth)
    
    return node


In [9]:
root = build_tree(X, y)
root.predicted_class, root.feature, root.threshold, root.left.predicted_class, root.right.predicted_class

(1, 'outlook', None, 1, 0)